MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Francesca Lazzeri Energy Demand Forecast Workbench workshop.

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Linear regression with recursive feature elimination

In [34]:
%matplotlib inline
import os
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
from azureml.core import Workspace, Dataset
from azureml.core.experiment import Experiment
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

This notebook shows how to train a linear regression model to create a forecast of future energy demand. In particular, the model will be trained to predict energy demand in period $t_{+1}$, one hour ahead of the current time period $t$. This is known as 'one-step' time series forecasting because we are predicting one period into the future.

In [3]:
WORKDIR = os.getcwd()
MODEL_NAME = "linear_regression"

In [5]:
ws = Workspace.from_config()

In [18]:
train_ds = Dataset.get_by_name(ws, name="capstone_data_processed")
print(train_ds.name, train_ds.version)

capstone_data_processed 2


In [23]:
train = (train_ds.to_pandas_dataframe()
                 .set_index(['data_index_'])
                 .loc[:'2021-05-31 00:00:00']
                 .reset_index(drop=True)
     # .drop(['load'], axis=1)
)

In [31]:
test =(train_ds.to_pandas_dataframe()
     .set_index(['data_index_'])
     .loc['2021-05-31 00:00:00':'2021-06-06 00:00:00']
     .reset_index(drop=True)
     )

Create design matrix - each column in this matrix represents a model feature and each row is a training example. We remove the *demand* and *timeStamp* variables as they are not model features.

In [32]:
lr = LinearRegression()
lr.fit(train.drop(['load'], axis=1), train["load"])
y_pred = lr.predict(test.drop(['load'], axis=1))

In [35]:
mean_absolute_percentage_error(test.load, y_pred)

0.017433209773943137

In [36]:

with open(os.path.join(WORKDIR, MODEL_NAME + '.pkl'), 'wb') as f:
    pickle.dump(lr, f)

In [ ]:
# lr_experiment = Experiment(ws, name="LR")
# run = lr_experiment.start_logging()

# run.log("dataset name", train_ds.name)
# run.log("dataset version", train_ds.version)

### Create predictive model pipeline

Here we use sklearn's Pipeline functionality to create a predictive model pipeline. For this model, the pipeline implements the following steps:
- **one-hot encode categorical variables** - this creates a feature for each unique value of a categorical feature. For example, the feature *dayofweek* has 7 unique values. This feature is split into 7 individual features dayofweek0, dayofweek1, ... , dayofweek6. The value of these features is 1 if the timeStamp corresponds to that day of the week, otherwise it is 0.
- **recursive feature elimination with cross validation (RFECV)** - it is often the case that some features add little predictive power to a model and may even make the model accuracy worse. Recursive feature elimination tests the model accuracy on increasingly smaller subsets of the features to identify the subset which produces the most accurate model. Cross validation is used to test each subset on multiple folds of the input data. The best model is that which achieves the lowest mean squared error averaged across the cross validation folds.
- **train final model** - the best model found in after the feature elimination process is used to train the final estimator on the whole dataset.

Identify indices for categorical columns for one hot encoding and create the OneHotEncoder:

In [ ]:
# cat_cols = ['hour', 'month', 'dayofweek']
# cat_cols_idx = [X.columns.get_loc(c) for c in X.columns if c in cat_cols]
# run.log_list("cat_cols", cat_cols)

In [ ]:
# preprocessor = ColumnTransformer([('encoder', OneHotEncoder(sparse=False), cat_cols_idx)], remainder='passthrough')

Create the linear regression model object:

In [15]:
regr = LinearRegression(fit_intercept=True)

For hyperparameter tuning and feature selection, cross validation will be performed using the training set. With time series forecasting, it is important that test data comes from a later time period than the training data. This also applies to each fold in cross validation. Therefore a time series split is used to create three folds for cross validation as illustrated below. Each time series plot represents a separate training/test split, with the training set coloured in blue and the test set coloured in red. Note that, even in the first split, the training data covers at least a full year so that the model can learn the annual seasonality of the demand.

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)

In [ ]:
demand_ts = train[['timeStamp', 'demand']].copy()
demand_ts.reset_index(drop=True, inplace=True)

for split_num, split_idx  in enumerate(tscv.split(demand_ts)):
    split_num = str(split_num)
    train_idx = split_idx[0]
    test_idx = split_idx[1]
    demand_ts['fold' + split_num] = "not used"
    demand_ts.loc[train_idx, 'fold' + split_num] = "train"
    demand_ts.loc[test_idx, 'fold' + split_num] = "test"

gs = gridspec.GridSpec(3,1)
fig = plt.figure(figsize=(15, 10), tight_layout=True)

ax = fig.add_subplot(gs[0])
ax.plot(demand_ts.loc[demand_ts['fold0']=="train", "timeStamp"], demand_ts.loc[demand_ts['fold0']=="train", "demand"], color='b')
ax.plot(demand_ts.loc[demand_ts['fold0']=="test", "timeStamp"], demand_ts.loc[demand_ts['fold0']=="test", "demand"], 'r')
ax.plot(demand_ts.loc[demand_ts['fold0']=="not used", "timeStamp"], demand_ts.loc[demand_ts['fold0']=="not used", "demand"], 'w')

ax = fig.add_subplot(gs[1], sharex=ax)
plt.plot(demand_ts.loc[demand_ts['fold1']=="train", "timeStamp"], demand_ts.loc[demand_ts['fold1']=="train", "demand"], 'b')
plt.plot(demand_ts.loc[demand_ts['fold1']=="test", "timeStamp"], demand_ts.loc[demand_ts['fold1']=="test", "demand"], 'r')
plt.plot(demand_ts.loc[demand_ts['fold1']=="not used", "timeStamp"], demand_ts.loc[demand_ts['fold1']=="not used", "demand"], 'w')

ax = fig.add_subplot(gs[2], sharex=ax)
plt.plot(demand_ts.loc[demand_ts['fold2']=="train", "timeStamp"], demand_ts.loc[demand_ts['fold2']=="train", "demand"], 'b')
plt.plot(demand_ts.loc[demand_ts['fold2']=="test", "timeStamp"], demand_ts.loc[demand_ts['fold2']=="test", "demand"], 'r')
plt.plot(demand_ts.loc[demand_ts['fold2']=="not used", "timeStamp"], demand_ts.loc[demand_ts['fold2']=="not used", "demand"], 'w')
plt.show()

Create the RFECV object. Note the metric for evaluating the model on each fold is the negative mean squared error. The best model is that which maximises this metric.

In [ ]:
regr_cv = RFECV(estimator=regr,
             cv=tscv,
             scoring='neg_mean_squared_error',
             verbose=2,
             n_jobs=-1)

Create the model pipeline object.

In [ ]:
regr_pipe = Pipeline([('preprocessor', preprocessor), ('rfecv', regr_cv)])

Train the model pipeline. This should only take a few seconds.

In [ ]:
regr_pipe.fit(X, y=train['demand'])

In [ ]:
run.log("pipeline steps", regr_pipe.named_steps)

Save the trained model pipeline object.

In [ ]:
with open(os.path.join(WORKDIR, MODEL_NAME + '.pkl'), 'wb') as f:
    pickle.dump(regr_pipe, f)

### Explore cross validation results

Best CV negative mean squared error:

In [ ]:
run.log("best CV negMSE", max(regr_pipe.named_steps['rfecv'].grid_scores_))

Plot the cross validation errors with each subset of features. The chart shows that most features are useful to the model. However, the error gets significantly worse when there are 44 features or less in the model.

In [ ]:
cv_results = pd.DataFrame.from_dict({'cv_score': regr_pipe.named_steps['rfecv'].grid_scores_})
cv_results['mean_squared_error'] = cv_results['cv_score']
plt.figure(figsize=(15, 5))
plt.plot(cv_results.index, cv_results['mean_squared_error'])
plt.xlabel('number of features')
plt.title('CV negative mean squared error')
run.log_image("CV errors plot", plot=plt)
plt.show()


Number of features selected:

In [ ]:
regr_pipe.named_steps['rfecv'].n_features_

Identify supported features after selection process:

In [ ]:
def get_onehot_cols(X):
    X_dummy_cols = list(pd.get_dummies(X.copy()[cat_cols], columns=cat_cols).columns)
    other_cols = list(X.columns.drop(cat_cols))
    return X_dummy_cols + other_cols

supported_features = pd.DataFrame.from_dict(
    {'feature':get_onehot_cols(X), 
     'supported':regr_pipe.named_steps['rfecv'].support_}
)
supported_features

Inspect model coefficients for each included feature. The magnitude and direction of the coefficients indicates the effect the features has on the model.

In [ ]:
coefs = supported_features.loc[supported_features['supported'], ].copy()
coefs['coefficients'] = regr_pipe.named_steps['rfecv'].estimator_.coef_
coefs.plot.bar('feature', 'coefficients', figsize=(15, 3), legend=False)
run.log_image("LR coefs per feature", plot=plt)
plt.show()

In [ ]:
run.complete()